In [23]:
# Importing necessary libraries
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential
from tensorflow.keras.utils import plot_model
import numpy as np
import matplotlib.pyplot as plt
import os
import random

# Libraries for TensorFlow
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras import models, layers
from tensorflow import keras

# Library for Transfer Learning
from tensorflow.keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
# Setting the image size for resizing
image_size = 28


def read_image(class_path, image_file):
    """
    Read and preprocess an image from a specified path.

    Parameters:
    - class_path: Path to the directory containing the image
    - image_file: Filename of the image

    Returns:
    - image: Preprocessed and resized grayscale image (2D numpy array)
    """
    image_path = os.path.join(class_path, image_file)
    image =  cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (image_size, image_size))
    return image


# Creating empty lists to store class labels and images
class_labels_list = []
images_list = []

# Path to the dataset directory
data_directory = './dataset/Validation/'

# List of class directories in the dataset
# directories = os.listdir(data_directory)
directories = ['A','B','C','D','E','F','G']


# Loop through each class directory
for class_label, class_name in enumerate(directories):
    class_path = os.path.join(data_directory, class_name)
    
    # Loop through each image file in the class directory
    for image_file in os.listdir(class_path):
        # Check if the file is a JPEG image
        if image_file.endswith('.jpg'):
            # Read and preprocess the image
            image = read_image(class_path, image_file)
            
            # Append the image and its corresponding class label to the lists
            images_list.append(image)
            class_labels_list.append(class_label)

# Separate data into feature vectors (X) and class labels (y)
data_x = images_list
data_y = class_labels_list

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, 
                                                    test_size=0.25, 
                                                    random_state=13)



In [24]:
# Convert the images into 3 channels as MNIST images are Black and White so have 1 channel

X_train = np.dstack([X_train] * 3)
X_test = np.dstack([X_test] * 3)

# Reshape images as per the tensor format required by tensorflow

X_train = X_train.reshape(-1, 28, 28, 3)
X_test = X_test.reshape (-1, 28, 28, 3)

# Resize the images 48*48 as required by VGG16

import keras.utils as image

X_train = np.asarray([image.img_to_array(image.array_to_img(im, scale=False).resize((48,48))) for im in X_train])
X_test = np.asarray([image.img_to_array(image.array_to_img(im, scale=False).resize((48,48))) for im in X_test])
#train_x = preprocess_input(x)
X_train.shape, X_test.shape

((1696, 48, 48, 3), (566, 48, 48, 3))

In [25]:

# initializing model with weights='imagenet'i.e. we are carring its original weights
model_vgg16=VGG16(weights='imagenet')


input_layer=layers.Input(shape=(48,48,3))

model_vgg16=VGG16(weights='imagenet',input_tensor=input_layer,include_top=False)

last_layer=model_vgg16.output 

flatten=layers.Flatten()(last_layer) 


layers.Dense(100,activation='relu')(flatten)
layers.Dense(100,activation='relu')(flatten)
layers.Dense(100,activation='relu')(flatten)

output_layer=layers.Dense(10,activation='softmax')(flatten)

model=models.Model(inputs=input_layer,outputs=output_layer)


# Freezing
# print("We are making all the layers intrainable except the last layer. \n")
# for layer in model.layers[:-1]:
#     layer.trainable=False


model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

history = model.fit(X_train,y_train,epochs=20,batch_size=128,verbose=True,validation_data=(X_test,y_test))


ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, (<class 'list'> containing values of types {"<class 'int'>"})